<a href="https://colab.research.google.com/github/MadhuReddy001/Image-Captioning/blob/main/Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 24.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [2]:
!pip install imbalanced-learn

In [3]:
!pip install protobuf==3.20.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.25.0 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.8.0 which is incompatible.


In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2025-01-06 17:06:42--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0 [following]
--2025-01-06 17:06:43--  https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbd1a8ca248ba30da21c4872027.dl.dropboxusercontent.com/zip_download_get/CFHlSms2otiDreUBdBflO2E7rc3tw3P0WL1CbqzRnYTsYx7qkSLinH07c2N9jpKQxoSso8Tvc5f-jinQUs4t2bzZ6SoR7qWfc6WhUH8VhZuVvg# [following]
--2025-01-06 17:06:44--  https://ucbd1a8ca248ba30da21c4872027.dl.dropboxusercontent.com/z

In [ ]:
import numpy as np
import pandas as pd
import gensim.downloader as api
import nltk

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from imblearn.over_sampling import RandomOverSampler

In [ ]:
filter = pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0', sep=',', encoding='latin-1', header=1)

In [ ]:
filter.drop(index=[1306122, 1306123], inplace=True)

In [ ]:
filter.drop(columns=['BN'], inplace=True)

In [ ]:
word2vec_model = api.load("word2vec-google-news-300")
embedding_index = {}
for word in word2vec_model.key_to_index:
    embedding_index[word] = word2vec_model[word]

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Embedding for 'example': [ 2.05078125e-01  7.85827637e-04  3.54003906e-02  1.00585938e-01
 -5.44433594e-02  1.53320312e-01  2.55859375e-01 -2.18750000e-01
 -3.31115723e-03  2.09960938e-01 -2.07031250e-01  1.77001953e-02
  4.29687500e-02 -2.01171875e-01 -1.57226562e-01  1.88476562e-01
 -3.73535156e-02  2.36816406e-02 -2.63671875e-01 -1.33789062e-01
  2.23632812e-01  2.05078125e-01 -5.83496094e-02 -3.11279297e-02
  4.92095947e-04  2.36328125e-01  1.16699219e-01  4.24804688e-02
 -1.33789062e-01  1.84570312e-01  5.02929688e-02 -6.00585938e-02
 -6.22558594e-02  7.61718750e-02  1.48437500e-01  6.10351562e-02
  6.39648438e-02 -2.73437500e-01  1.48437500e-01  8.15429688e-02
  1.57226562e-01 -2.63671875e-02 -1.10839844e-01  3.24707031e-02
 -6.93359375e-02 -3.29589844e-02 -1.34765625e-01  4.32128906e-02
 -1.42578125e-01 -2.50000000e-01  9.86328125e-02 -1.10839844e-01
 -6.98242188e-02 -2.46093750e-01  1.6503906

In [ ]:
df = pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0', sep=',', encoding='latin-1', header=1)

X = df['question_text']
Y = df['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

((1044899,), (261225,), (1044899,), (261225,))

In [ ]:
class_weight = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train.dropna().astype(int)),
    y=y_train.dropna().astype(int)
)

[0.53298834 8.07843425]


In [ ]:
class_weight_dict = dict(enumerate(class_weight))

{0: 0.5329883445127395, 1: 8.078434252845126}

In [ ]:
x

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sent_lens = [len(word_tokenize(str(x))) for x in x_train]

412


In [ ]:
max_len = 44

tk = Tokenizer(char_level=False, split=' ')

x_train = x_train.astype(str)
x_test = x_test.astype(str)


x_train = x_train.apply(lambda x: x.lower() if isinstance(x, str) else '')
x_test = x_test.apply(lambda x: x.lower() if isinstance(x, str) else '')

tk.fit_on_texts(x_train)

seq_train = tk.texts_to_sequences(x_train)
seq_test = tk.texts_to_sequences(x_test)

vocab_size = len(tk.word_index)

seq_train_matrix = sequence.pad_sequences(seq_train, maxlen=max_len)
seq_test_matrix = sequence.pad_sequences(seq_test, maxlen=max_len)

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'way': 78,
 'out': 79,
 'am': 80,
 'know': 81,
 'b

In [ ]:
word2vec_model = api.load("word2vec-google-news-300")

embedding_dim = 300
vocab_size = len(tk.word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tk.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

Number of rows in the embedding matrix: 196190


In [ ]:
inputs = Input(name='text_input', shape=[max_len])

embed = Embedding(len(embedding_matrix), 300,
                  input_length=max_len, mask_zero=True,
                  weights=[embedding_matrix], trainable=False)(inputs)

lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25, activation='relu')(drop3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1, activation='sigmoid')(drop4)

model = Model(inputs=inputs, outputs=final_layer)

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.keras' # Changed .h5 to .keras
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
max_len = 44
EMBEDDING_DIM = 100
vocab_size = len(tk.word_index)

inputs = Input(name='text_input', shape=[max_len])

embed = Embedding(vocab_size + 1, EMBEDDING_DIM,
                  input_length=max_len, mask_zero=True,
                  trainable=False)(inputs)

lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25, activation='relu')(drop3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1, activation='sigmoid')(drop4)

model = Model(inputs=inputs, outputs=final_layer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 44, 100)           19619000  
                                                                 
 lstm_1 (LSTM)               (None, 512)               1255424   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               3289

In [ ]:
print(y_train.isna().sum())

non_nan_indices = ~y_train.isna()
seq_train_matrix = seq_train_matrix[non_nan_indices]
y_train = y_train[non_nan_indices]

ros = RandomOverSampler(random_state=42)
seq_train_matrix_resampled, y_train_resampled = ros.fit_resample(
    seq_train_matrix, y_train
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    seq_train_matrix_resampled,
    y_train_resampled,
    epochs=15,
    batch_size=1000,
    validation_data=(seq_test_matrix, y_test),
    callbacks=[earlystop, checkpoint]
)

2
Epoch 1/15


In [ ]:
best_model = load_model('/content/weights-04-0.2007.keras')

In [ ]:
best_model_weights = load_model('/content/weights-04-0.2007.keras').get_weights()

inputs = Input(name='text_input', shape=[max_len])
embed = Embedding(vocab_size + 1, 300, input_length=max_len, mask_zero=True,
                  weights=[embedding_matrix], trainable=False)(inputs)
lstm_layer1 = LSTM(512, use_cudnn=False)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25, activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1, activation='sigmoid')(drop4)

recreated_model = Model(inputs=inputs, outputs=final_layer)

recreated_model.set_weights(best_model_weights)

p = recreated_model.predict(seq_test_matrix)
roc_auc_score(y_test, p)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step


0.9644149735158491